# Capstone Project - The Battle of the Neighborhoods (Week 2)

Applied Data Science Capstone by IBM/Coursera

##### Table of contents
- Introduction: Business Problem

- Data

- Methodology

- Results and Discussion

- Conclusion

##### Introduction: Business Problem 

In this project, i am going to analyze the cities of Lisbon and Oporto, to determine between them, which one is more homogeneous in terms of most common venues, in order to help tourists decide which one they are going to visit in a trip to Portugal, and might feel more comfortable doing so

I will apply some Data Science techniques, such as Clustering by K-Means, and visualization tools like Folium, to help understand better the purpose of the project. A final discussion and conclusion will determine which city is more promising

##### Data 

For this project, I will be using:

- Data from Wikipedia, about neighborhoods and their geolocation, from the cities of Lisbon and Oporto, assembled and saved as an excel spreadsheet and then loaded into the model as a data frame


- Afterwards will extract the venues from those locations from Foursquare API


- Then develop the model with Python tools, applying cluster analysis and data visualization techniques, to obtain the final conclusions



##### Methodology

First, I´m going to scrape Wikipedia to build the Dataframes. As Lisboa and Porto don´t have a pre-prepared table of the Neighborhoods and their geolocation, it´s necessary to find the list of neighborhoods (Freguesias) and get the geolocation of their centroid. This is very simple, it can be done with the help of Wikipedia and Google Maps. The result is going to be two excel Spreadsheet, one for Lisboa and another for Porto

Second step will be to get the most 10 common venues for which neighborhood, with the help of Foursquare API and Python coding. This is important because if we consider venues as services (coffee, restaurant, library, museum), these are all important services to help tourists enjoy their site-seeing in each city, and with the following step, it´s going to be analyzed how the cities form clusters with similar venues more relevant to tourists

In third and final step I´m going to focus on creating the clusters with K-means and showing a visualization of them in each city, and come with a conclusion with a simple breakdown of the characteristics of each city

In [30]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install folium geopy # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium==0.5.0
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Beginning with Oporto

In [31]:
CLIENT_ID = 'VQFFLCEOOKT4UIOBL5WFKF31W2FIBDJZXG0ZCHMJJL5FLJKM' # your Foursquare ID
CLIENT_SECRET = 'MIMZV12UEKRNV31GECV2J0MNOXLWQRPROM24UCVVS2YPMEX2' # your Foursquare Secret
VERSION = '20200303' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VQFFLCEOOKT4UIOBL5WFKF31W2FIBDJZXG0ZCHMJJL5FLJKM
CLIENT_SECRET:MIMZV12UEKRNV31GECV2J0MNOXLWQRPROM24UCVVS2YPMEX2


In [32]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_d3ac577184a14e229098790b0b8b3136 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='tWJ-4omEmsQqsmFWhCI0d9hLDMw4BuX2jkpeeB1kcp0f',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_d3ac577184a14e229098790b0b8b3136.get_object(Bucket='capstone-donotdelete-pr-ciey9vvu9fdqkb',Key='Porto.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

porto_pd = pd.read_excel(body)
porto_pd


,Neighborhood,Latitude,Longitude
0,Aldoar,41.173146,-8.668896
1,Bonfim,41.152636,-8.596838
2,Campanha,41.160200,-8.576296
3,Cedofeita,41.159379,-8.620678
4,Foz do Douro,41.152790,-8.670620
5,Lordelo do Ouro,41.154250,-8.650091
6,Massarelos,41.152636,-8.631807
7,Miragaia,41.146517,-8.620804
8,Nevogilde,41.164288,-8.681636
9,Paranhos,41.174265,-8.606006


In [33]:
address = 'Porto, Portugal'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Porto are {}, {}.'.format(latitude, longitude))

# create map of Porto using latitude and longitude values
map_porto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(porto_pd['Latitude'], porto_pd['Longitude'], porto_pd['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_porto)  
    
map_porto

The geograpical coordinate of Porto are 41.1494512, -8.6107884.


In [34]:
neighborhood_latitude = porto_pd.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = porto_pd.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = porto_pd.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

Latitude and longitude values of Aldoar are 41.173146, -8.668896.


'https://api.foursquare.com/v2/venues/explore?&client_id=VQFFLCEOOKT4UIOBL5WFKF31W2FIBDJZXG0ZCHMJJL5FLJKM&client_secret=MIMZV12UEKRNV31GECV2J0MNOXLWQRPROM24UCVVS2YPMEX2&v=20200303&ll=41.173146,-8.668896&radius=500&limit=100'

In [35]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e83cc256001fe001b877405'},
 'response': {'headerLocation': 'Porto',
  'headerFullLocation': 'Porto',
  'headerLocationGranularity': 'city',
  'totalResults': 6,
  'suggestedBounds': {'ne': {'lat': 41.177646004500005,
    'lng': -8.662928865732251},
   'sw': {'lat': 41.1686459955, 'lng': -8.67486313426775}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '500ac817e4b0a62ff46a04f1',
       'name': 'Soundwich',
       'location': {'address': 'Av. do Parque 595',
        'crossStreet': 'Beco das Carreiras 64/65',
        'lat': 41.17031384203106,
        'lng': -8.671641657687033,
        'labeledLatLngs': [{'label': 'display',
          'lat': 41.17031384203106,
          'lng': -8.671641657687033}],
        'distance': 390,
        'po

In [36]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

6 venues were returned by Foursquare.


In [37]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [38]:


porto_venues = getNearbyVenues(names=porto_pd['Neighborhood'],
                                   latitudes=porto_pd['Latitude'],
                                   longitudes=porto_pd['Longitude']
                                  )

Aldoar
Bonfim
Campanha
Cedofeita
Foz do Douro
Lordelo do Ouro
Massarelos
Miragaia
Nevogilde
Paranhos
Ramalde
Santo Ildefonso
Sao Nicolau
Se
Vitoria


In [39]:
print(porto_venues.shape)
porto_venues.head()

(574, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Aldoar,41.173146,-8.668896,Soundwich,41.170314,-8.671642,Sandwich Place
1,Aldoar,41.173146,-8.668896,Meet Parque,41.170065,-8.672510,Café
2,Aldoar,41.173146,-8.668896,Café do Parque da Cidade,41.170143,-8.672567,Café
3,Aldoar,41.173146,-8.668896,iLoja,41.169605,-8.671586,IT Services
4,Aldoar,41.173146,-8.668896,Açafrão,41.175842,-8.665793,Café


In [40]:
porto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Aldoar,6,6,6,6,6,6
Bonfim,15,15,15,15,15,15
Campanha,4,4,4,4,4,4
Cedofeita,34,34,34,34,34,34
Foz do Douro,23,23,23,23,23,23
Lordelo do Ouro,12,12,12,12,12,12
Massarelos,43,43,43,43,43,43
Miragaia,80,80,80,80,80,80
Nevogilde,8,8,8,8,8,8


In [41]:
print('There are {} uniques categories.'.format(len(porto_venues['Venue Category'].unique())))

There are 125 uniques categories.


In [42]:
# one hot encoding
porto_onehot = pd.get_dummies(porto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
porto_onehot['Neighborhood'] = porto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [porto_onehot.columns[-1]] + list(porto_onehot.columns[:-1])
porto_onehot = porto_onehot[fixed_columns]

porto_onehot.head()

,Yoga Studio,African Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bike Rental / Bike Share,Bike Shop,Bistro,Boarding House,Bookstore,Boutique,Breakfast Spot,Brewery,Bridge,Building,Burger Joint,Bus Station,Café,Candy Store,Cemetery,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Cosmetics Shop,Creperie,Dessert Shop,Diner,Electronics Store,Event Space,Fast Food Restaurant,Flower Shop,Food Court,Furniture / Home Store,Garden,Gas Station,Gastropub,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Historic Site,History Museum,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Karaoke Bar,Liquor Store,Lounge,Market,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Museum,Music Venue,Neighborhood,Nightclub,Nightlife Spot,Noodle House,Other Great Outdoors,Palace,Park,Pastry Shop,Pedestrian Plaza,Pet Store,Pharmacy,Pizza Place,Plaza,Portuguese Restaurant,Ramen Restaurant,Resort,Restaurant,Road,Roof Deck,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shop & Service,Shopping Mall,Snack Place,Soccer Field,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Syrian Restaurant,Tapas Restaurant,Taverna,Tea Room,Tennis Court,Tennis Stadium,Theater,Train Station,Vegetarian / Vegan Restaurant,Waterfront,Wine Bar,Wine Shop,Winery
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Aldoar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Aldoar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Aldoar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Aldoar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Aldoar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [43]:
porto_onehot.shape

(574, 125)

In [44]:
porto_grouped = porto_onehot.groupby('Neighborhood').mean().reset_index()
porto_grouped

,Neighborhood,Yoga Studio,African Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bike Rental / Bike Share,Bike Shop,Bistro,Boarding House,Bookstore,Boutique,Breakfast Spot,Brewery,Bridge,Building,Burger Joint,Bus Station,Café,Candy Store,Cemetery,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Cosmetics Shop,Creperie,Dessert Shop,Diner,Electronics Store,Event Space,Fast Food Restaurant,Flower Shop,Food Court,Furniture / Home Store,Garden,Gas Station,Gastropub,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Historic Site,History Museum,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Karaoke Bar,Liquor Store,Lounge,Market,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Museum,Music Venue,Nightclub,Nightlife Spot,Noodle House,Other Great Outdoors,Palace,Park,Pastry Shop,Pedestrian Plaza,Pet Store,Pharmacy,Pizza Place,Plaza,Portuguese Restaurant,Ramen Restaurant,Resort,Restaurant,Road,Roof Deck,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shop & Service,Shopping Mall,Snack Place,Soccer Field,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Syrian Restaurant,Tapas Restaurant,Taverna,Tea Room,Tennis Court,Tennis Stadium,Theater,Train Station,Vegetarian / Vegan Restaurant,Waterfront,Wine Bar,Wine Shop,Winery
0,Aldoar,0.000000,0.0000,0.000000,0.00,0.000000,0.0000,0.0000,0.000000,0.0000,0.000000,0.000000,0.00,0.0000,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.500000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.166667,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.0000,0.000000,0.0000,0.00,0.000000,0.00,0.0000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.0000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.166667,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00
1,Bonfim,0.000000,0.0000,0.000000,0.00,0.000000,0.0000,0.0000,0.000000,0.0000,0.000000,0.000000,0.00,0.0000,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.066667,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.00000,0.00,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.066667,0.000000,0.000000,0.00,0.0000,0.000000,0.0000,0.066667,0.000000,0.200000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.0000,0.000000,0.0000,0.00,0.000000,0.00,0.0000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.0000,0.0000,0.000000,0.0,0.000000,0.000000,0.066667,0.066667,0.000000,0.000000,0.200000,0.0000,0.000000,0.000000,0.00000,0.066667,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.0000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.066667,0.00,0.000000,0.00,0.00
2,Campanha,0.000000,0.0000,0.000000,0.00,0.000000,0.0000,0.0000,0.000000,0.0000,0.000000,0.000000,0.00,0.0000,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.000000,0.0

In [45]:
porto_grouped.shape

(15, 125)

In [26]:
num_top_venues = 5

for hood in porto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = porto_grouped[porto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Aldoar----
            venue  freq
0            Café  0.50
1     IT Services  0.17
2  Sandwich Place  0.17
3     Flower Shop  0.17
4           Plaza  0.00


----Bonfim----
                   venue  freq
0             Restaurant  0.20
1                  Hotel  0.20
2  Portuguese Restaurant  0.07
3          Grocery Store  0.07
4                  Plaza  0.07


----Campanha----
                venue  freq
0  Miscellaneous Shop  0.25
1       Metro Station  0.25
2       Grocery Store  0.25
3                Park  0.25
4         Yoga Studio  0.00


----Cedofeita----
                   venue  freq
0                   Café  0.15
1             Restaurant  0.09
2                 Bakery  0.09
3     Italian Restaurant  0.06
4  Portuguese Restaurant  0.06


----Foz do Douro----
                   venue  freq
0             Restaurant  0.13
1  Portuguese Restaurant  0.09
2                   Café  0.09
3            Yoga Studio  0.04
4       Tapas Restaurant  0.04


----Lordelo do Ouro----
          

In [46]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [47]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
porto_pd_venues_sorted = pd.DataFrame(columns=columns)
porto_pd_venues_sorted['Neighborhood'] = porto_grouped['Neighborhood']

for ind in np.arange(porto_grouped.shape[0]):
    porto_pd_venues_sorted.iloc[ind, 1:] = return_most_common_venues(porto_grouped.iloc[ind, :], num_top_venues)

porto_pd_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aldoar,Café,IT Services,Sandwich Place,Flower Shop,Winery,Food Court,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Cosmetics Shop
1,Bonfim,Hotel,Restaurant,Hostel,Portuguese Restaurant,Seafood Restaurant,Supermarket,Grocery Store,Coffee Shop,Plaza,Vegetarian / Vegan Restaurant
2,Campanha,Grocery Store,Park,Metro Station,Miscellaneous Shop,Comfort Food Restaurant,Coffee Shop,Cosmetics Shop,Creperie,Food Court,Dessert Shop
3,Cedofeita,Café,Restaurant,Bakery,Portuguese Restaurant,Hotel,Italian Restaurant,Bar,Pharmacy,Steakhouse,Diner
4,Foz do Douro,Restaurant,Café,Portuguese Restaurant,Yoga Studio,Sushi Restaurant,Coffee Shop,Pharmacy,Pizza Place,Plaza,Modern European Restaurant


In [48]:
# set number of clusters
kclusters = 7

porto_grouped_clustering = porto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(porto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 1, 0, 0, 4, 4, 4, 5, 6], dtype=int32)

In [54]:
porto_pd_venues_sorted=porto_pd_venues_sorted.drop('Cluster Labels',1)

In [49]:
porto_pd_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [50]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)



porto_merged = porto_pd

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
porto_merged = porto_merged.join(porto_pd_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


In [51]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(porto_merged['Latitude'], porto_merged['Longitude'], porto_merged['Neighborhood'], porto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Analyzing Lisboa

In [52]:
# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_d3ac577184a14e229098790b0b8b3136 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='tWJ-4omEmsQqsmFWhCI0d9hLDMw4BuX2jkpeeB1kcp0f',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_d3ac577184a14e229098790b0b8b3136.get_object(Bucket='capstone-donotdelete-pr-ciey9vvu9fdqkb',Key='Lisboa.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

lisboa_pd = pd.read_excel(body)
lisboa_pd=pd.DataFrame(lisboa_pd)
lisboa_pd.head()

,Neighborhood,Latitude,Longitude
0,Ajuda,38.708597,-9.198953
1,Alcantara,38.708525,-9.179814
2,Alvalade,38.746944,-9.136111
3,Areeiro,38.740278,-9.128056
4,Arroios,38.728889,-9.138889


In [53]:
address = 'Lisboa, Portugal'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Lisboa are {}, {}.'.format(latitude, longitude))

# create map of Lisboa using latitude and longitude values
map_lisboa = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(lisboa_pd['Latitude'],lisboa_pd['Longitude'], lisboa_pd['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_lisboa)   
    
map_lisboa

The geograpical coordinate of Lisboa are 38.7077507, -9.1365919.


In [54]:
neighborhood_latitude = lisboa_pd.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = lisboa_pd.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = lisboa_pd.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

Latitude and longitude values of Ajuda are 38.708597, -9.198953.


'https://api.foursquare.com/v2/venues/explore?&client_id=VQFFLCEOOKT4UIOBL5WFKF31W2FIBDJZXG0ZCHMJJL5FLJKM&client_secret=MIMZV12UEKRNV31GECV2J0MNOXLWQRPROM24UCVVS2YPMEX2&v=20200303&ll=38.708597,-9.198953&radius=500&limit=100'

In [55]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e83cbabbae9a2001bcfe85a'},
 'response': {'headerLocation': 'Ajuda',
  'headerFullLocation': 'Ajuda, Lisbon',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 6,
  'suggestedBounds': {'ne': {'lat': 38.7130970045, 'lng': -9.193197019091107},
   'sw': {'lat': 38.704096995499995, 'lng': -9.204708980908892}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b0588a3f964a5207bd122e3',
       'name': 'Palácio Nacional da Ajuda',
       'location': {'address': 'Largo da Ajuda',
        'lat': 38.707653,
        'lng': -9.197758,
        'labeledLatLngs': [{'label': 'display',
          'lat': 38.707653,
          'lng': -9.197758}],
        'distance': 147,
        'postalCode': '1349-021',
        'cc': 'PT',
        'city': 

In [56]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

6 venues were returned by Foursquare.


In [57]:


lisboa_venues = getNearbyVenues(names=lisboa_pd['Neighborhood'],
                                   latitudes=lisboa_pd['Latitude'],
                                   longitudes=lisboa_pd['Longitude']
                                  )

Ajuda
Alcantara
Alvalade
Areeiro
Arroios
Avenidas Novas
Beato
Belem
Benfica
Campo de Ourique
Campolide
Carnide
Estrela
Lumiar
Marvila
Misericordia
Olivais
Parque das Nacoes
Penha de Franca
Santa Clara
Santa Maria Maior
Santo Antonio
Sao Domingos de Benfica
Sao Vicente


In [58]:
print(lisboa_venues.shape)
lisboa_venues.head()

(949, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Ajuda,38.708597,-9.198953,Palácio Nacional da Ajuda,38.707653,-9.197758,Historic Site
1,Ajuda,38.708597,-9.198953,Restaurante Andorinhas,38.704911,-9.199349,Restaurant
2,Ajuda,38.708597,-9.198953,Jardim Botânico da Ajuda,38.706430,-9.201222,Botanical Garden
3,Ajuda,38.708597,-9.198953,Páteo Alfacinha,38.706537,-9.194202,Restaurant
4,Ajuda,38.708597,-9.198953,Estufa Real,38.706840,-9.201975,Restaurant


In [59]:
lisboa_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Ajuda,6,6,6,6,6,6
Alcantara,10,10,10,10,10,10
Alvalade,43,43,43,43,43,43
Areeiro,27,27,27,27,27,27
Arroios,36,36,36,36,36,36
Avenidas Novas,77,77,77,77,77,77
Beato,8,8,8,8,8,8
Belem,53,53,53,53,53,53
Benfica,44,44,44,44,44,44


In [60]:
print('There are {} uniques categories.'.format(len(lisboa_venues['Venue Category'].unique())))

There are 173 uniques categories.


In [61]:
# one hot encoding
lisboa_onehot = pd.get_dummies(lisboa_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
lisboa_onehot['Neighborhood'] = lisboa_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [lisboa_onehot.columns[-1]] + list(lisboa_onehot.columns[:-1])
lisboa_onehot = lisboa_onehot[fixed_columns]

lisboa_onehot.head()

,Neighborhood,Accessories Store,African Restaurant,Amphitheater,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Bistro,Board Shop,Boarding House,Bookstore,Botanical Garden,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Bus Station,Business Service,Cable Car,Cafeteria,Café,Candy Store,Cantonese Restaurant,Capitol Building,Casino,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Creperie,Cultural Center,Dance Studio,Dim Sum Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Lebanese Restaurant,Liquor Store,Lounge,Market,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motorcycle Shop,Movie Theater,Nightclub,Noodle House,Organic Grocery,Other Nightlife,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Swiss Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Tibetan Restaurant,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop
0,Ajuda,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Ajuda,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Ajuda,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Ajuda,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Ajuda,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [62]:
lisboa_onehot.shape
lisboa_grouped = lisboa_onehot.groupby('Neighborhood').mean().reset_index()
lisboa_grouped

,Neighborhood,Accessories Store,African Restaurant,Amphitheater,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Bistro,Board Shop,Boarding House,Bookstore,Botanical Garden,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Bus Station,Business Service,Cable Car,Cafeteria,Café,Candy Store,Cantonese Restaurant,Capitol Building,Casino,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Creperie,Cultural Center,Dance Studio,Dim Sum Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Lebanese Restaurant,Liquor Store,Lounge,Market,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motorcycle Shop,Movie Theater,Nightclub,Noodle House,Organic Grocery,Other Nightlife,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Swiss Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Tibetan Restaurant,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop
0,Ajuda,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.0,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.166667,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.0,0.00,0.00,0.00,0.000000,0.00,0.000,0.000000,0.000000,0.00,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Alcantara,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.1,0.000000,0.00,0.000000,0.00

In [63]:
num_top_venues = 5

for hood in lisboa_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = lisboa_grouped[lisboa_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Ajuda----
               venue  freq
0         Restaurant  0.50
1      Historic Site  0.17
2   Botanical Garden  0.17
3             Bakery  0.17
4  Accessories Store  0.00


----Alcantara----
                         venue  freq
0     Mediterranean Restaurant   0.2
1           Basketball Stadium   0.1
2               Soccer Stadium   0.1
3                        Plaza   0.1
4  Eastern European Restaurant   0.1


----Alvalade----
                   venue  freq
0  Portuguese Restaurant  0.12
1                   Café  0.09
2              Bookstore  0.07
3            Coffee Shop  0.05
4         Ice Cream Shop  0.05


----Areeiro----
                   venue  freq
0                  Hotel  0.15
1  Portuguese Restaurant  0.15
2      Electronics Store  0.11
3           Tennis Court  0.04
4            Pizza Place  0.04


----Arroios----
                           venue  freq
0                          Hotel  0.11
1          Portuguese Restaurant  0.11
2              Indian Restaurant  0.08

In [64]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
lisboa_pd_venues_sorted = pd.DataFrame(columns=columns)
lisboa_pd_venues_sorted['Neighborhood'] = lisboa_grouped['Neighborhood']

for ind in np.arange(lisboa_grouped.shape[0]):
    lisboa_pd_venues_sorted.iloc[ind, 1:] = return_most_common_venues(lisboa_grouped.iloc[ind, :], num_top_venues)

lisboa_pd_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ajuda,Restaurant,Botanical Garden,Historic Site,Bakery,Empanada Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish & Chips Shop
1,Alcantara,Mediterranean Restaurant,Plaza,Basketball Stadium,Eastern European Restaurant,Restaurant,Café,Soccer Stadium,Supermarket,Market,Garden
2,Alvalade,Portuguese Restaurant,Café,Bookstore,Hotel,Coffee Shop,Bakery,Ice Cream Shop,Electronics Store,BBQ Joint,Salon / Barbershop
3,Areeiro,Hotel,Portuguese Restaurant,Electronics Store,Wine Shop,Stadium,Italian Restaurant,Fountain,Flower Shop,Coffee Shop,Pizza Place
4,Arroios,Hotel,Portuguese Restaurant,Indian Restaurant,BBQ Joint,Vegetarian / Vegan Restaurant,Bakery,Hostel,Gym,Breakfast Spot,Seafood Restaurant


In [65]:
# set number of clusters
kclusters = 5

lisboa_grouped_clustering = lisboa_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(lisboa_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 3, 3, 3, 3, 2, 0, 3, 3], dtype=int32)

In [117]:
lisboa_pd_venues_sorted=porto_pd_venues_sorted.drop('Cluster Labels',1)

In [66]:
lisboa_pd_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [68]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

lisboa_merged = lisboa_pd

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
lisboa_merged = lisboa_merged.join(lisboa_pd_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

lisboa_merged['Cluster Labels'].isnull # check the last columns!

<bound method Series.isnull of 0     2
1     1
2     3
3     3
4     3
5     3
6     2
7     0
8     3
9     3
10    3
11    0
12    0
13    1
14    4
15    3
16    0
17    3
18    0
19    1
20    3
21    3
22    1
23    3
Name: Cluster Labels, dtype: int32>

In [69]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(lisboa_merged['Latitude'], lisboa_merged['Longitude'], lisboa_merged['Neighborhood'], lisboa_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

##### Results and Discussion 

First it´s mandatory to explain that the number of clusters for each city was tested with different possibilities, and that the actual number presented in this report it´s the one that best fits the venues locations, accordingly to my in-site knowledge of both cities, taking in account the time i lived and visited them. In fact the results show conclusions that most of Portuguese citizens already know, as what i wanted to prove is that with a scientific model and Data Science tools, i could prove what i managed to learn throughout my life about these wonderful Portuguese cities

We learned that the most common venues are coffees, restaurants and hotels, which suites the tourists fine, to help them in their travelling and accordingly to the model, Porto has seven clusters and Lisboa 5. 

For Porto, we can see that closer to the center of the city, more equal is the cluster, as there´s a big concentration of neighborhoods in the same cluster. And so it is in real-life. Porto has more similarities between those neighborhoods near to the center, and the outer ones are more dissimilar.
As for Lisbon, it has 5 clusters but they are more spreaded. The green cluster is more centered on downtown, but it has some neighborhoods like Parque das Nações and Benfica included, which are more in the surroundings. This finding also fits the reality because all these neighborhoods are rich in coffees and restaurants. Also there are other clusters that have members that aren´t near each other but they are similar neighborhoods. What we can detect about Lisboa is that is more diverse, showing different focus of growth and evolution of neighborhoods throughout the years, because as we can infer the growth of the city wasn´t always from the river to North


##### Conclusion

Finally, i can say that as Porto is smaller and more similar, tourists can have a better experience there, if they like to walk and spend more time in the same neighborhoods, experiencing the real life of Porto. On the other hand, Lisbon is bigger and more diverse, and with more to visit and experiment. In my experience, Lisbon is more exciting and cosmopolitan but it´s known that people from Porto have a good charisma. I guess it will depend on the kind of tourist to decide between both of them, but personally i would choose Lisboa as it also has some very old, and with lots of history neighborhoods, and with very friendly people